# DALLE-Server를 Colab에서 구동하기.

이용하기 쉬운 외부 가상환경인 Colab에서 DALLE-Server를 구동합니다.
Colab 을 상시 서버로 사용하기는 힘들기 때문에 단순히 Test 용도로 쓰는게 좋습니다.

## 필요 파일 설치

서버를 구동하기 위해 필요한 기본 패키지를 Colab 환경에 설치합니다.
의존성 및 안정적인 구동을 고려해 원작자가 추천하는 기본 패키지로 재설치하겠습니다.

In [1]:
!pip install Flask==1.1.2
!pip install Flask-Cors==3.0.9
!pip install Flask-RESTful==0.3.8
!pip install dalle-pytorch==0.14.3
!pip install tqdm
!pip install deepspeed==0.4.2
!pip install triton==0.4.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## 패키지 임포트

In [2]:
from flask import Flask, request, jsonify
from tqdm import tqdm
import numpy as np
from queue import Queue, Empty
from threading import Thread
import time

# torch

import torch

from einops import repeat

# vision imports

from PIL import Image

# dalle related classes and utils

from dalle_pytorch import VQGanVAE, DALLE
from dalle_pytorch.tokenizer import tokenizer

import base64
from io import BytesIO

## Flask App 생성

Flask App을 생성하고 초기화합니다.

In [3]:
app = Flask(__name__)

requests_queue = Queue()  # request queue.
REQUEST_BATCH_SIZE = 4  # max request size.
CHECK_INTERVAL = 0.1

## Model 불러오기

VQGAN 의 엔코더를 VAE로 불러옵니다.

In [4]:
# load model

vae = VQGanVAE(None, None)

Working with z of shape (1, 256, 16, 16) = 65536 dimensions.
loaded pretrained LPIPS loss from taming/modules/autoencoder/lpips/vgg.pth
VQLPIPSWithDiscriminator running with hinge loss.
Loaded VQGAN from /root/.cache/dalle/vqgan.1024.model.ckpt and /root/.cache/dalle/vqgan.1024.config.yml


## Checkpoint download

Pretrained 된 값을 가져오기 위해 Checkpoint를 가져오고, 이를 불러옵니다.
참고로 필요한 Checkpoint는 아래 위치에서 참고했습니다.

- model checkpoint : https://github.com/robvanvolt/DALLE-models/tree/main/models/taming_transformer

In [5]:
!wget --no-check-certificate https://www.dropbox.com/s/8mmgnromwoilpfm/16L_64HD_8H_512I_128T_cc12m_cc3m_3E.pt

--2022-07-01 16:22:46--  https://www.dropbox.com/s/8mmgnromwoilpfm/16L_64HD_8H_512I_128T_cc12m_cc3m_3E.pt
Resolving www.dropbox.com (www.dropbox.com)... 162.125.71.18, 2620:100:6021:18::a27d:4112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.71.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/8mmgnromwoilpfm/16L_64HD_8H_512I_128T_cc12m_cc3m_3E.pt [following]
--2022-07-01 16:22:46--  https://www.dropbox.com/s/raw/8mmgnromwoilpfm/16L_64HD_8H_512I_128T_cc12m_cc3m_3E.pt
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc82a66a98d30c6d20cd1becd837.dl.dropboxusercontent.com/cd/0/inline/BoTMzvSJy0HsNWjiI5NRAMOoFs9F3teEmOzgcIoocx3kvJAJ9yKhQKDtybg-PAQFYO2sYPTet2LPplJ9qp4MedNaobQXjjSXCoiwRehrGqLkU-lOb1Rr3_pIzIox9q8M6RHY-SQy7Q-NKTLR0nnM1q9P-QgeTxHACXA2nlXiow1rmA/file# [following]
--2022-07-01 16:22:47--  https://uc82a66a98d30c6d20cd1becd837.dl.dropboxusercontent.com

In [6]:
load_obj = torch.load("./16L_64HD_8H_512I_128T_cc12m_cc3m_3E.pt")
dalle_params, _, weights = load_obj.pop('hparams'), load_obj.pop('vae_params'), load_obj.pop('weights')
dalle_params.pop('vae', None)  # cleanup later

dalle = DALLE(vae=vae, **dalle_params).cuda()
dalle.load_state_dict(weights)
batch_size = 4
top_k = 0.9

## Image 생성

다운로드 받은 Checkpoint에 맞게 Image를 생성합니다.


In [7]:
def make_images(text_input, num_images):
    try:
        text = tokenizer.tokenize([text_input], dalle.text_seq_len).cuda()

        text = repeat(text, '() n -> b n', b=num_images)

        outputs = []
        for text_chunk in tqdm(text.split(batch_size), desc=f'generating images for - {text}'):
            output = dalle.generate_images(text_chunk, filter_thres=top_k)
            outputs.append(output)

        outputs = torch.cat(outputs)

        response = []

        for i, image in tqdm(enumerate(outputs), desc='saving images'):
            np_image = np.moveaxis(image.cpu().numpy(), 0, -1)
            formatted = (np_image * 255).astype('uint8')

            img = Image.fromarray(formatted)

            buffered = BytesIO()
            img.save(buffered, format="JPEG")
            img_str = base64.b64encode(buffered.getvalue()).decode("utf-8")
            response.append(img_str)

        return response

    except Exception as e:
        print('Error occur in script generating!', e)
        return jsonify({'Error': e}), 500

def handle_requests_by_batch():
    while True:
        request_batch = []

        while not (len(request_batch) >= REQUEST_BATCH_SIZE):
            try:
                request_batch.append(requests_queue.get(timeout=CHECK_INTERVAL))
            except Empty:
                continue

            for requests in request_batch:
                try:
                    requests["output"] = make_images(requests['input'][0], requests['input'][1])

                except Exception as e:
                    requests["output"] = e


In [8]:
handler = Thread(target=handle_requests_by_batch).start()

## Flask 블루 프린트 구성

생성된 Image를 Flask와 REST로 보낼 수 있는 router를 구성합니다.

In [9]:
@app.route('/generate', methods=['POST'])
def generate():
    if requests_queue.qsize() > REQUEST_BATCH_SIZE:
        return jsonify({'Error': 'Too Many Requests. Please try again later'}), 429

    try:
        args = []
        json_data = request.get_json()
        text_input = json_data["text"]
        num_images = json_data["num_images"]

        if num_images > 10:
            return jsonify({'Error': 'Too many images requested. Request no more than 10'}), 500

        args.append(text_input)
        args.append(num_images)

    except Exception as e:
        return jsonify({'Error': 'Invalid request'}), 500

    req = {'input': args}
    requests_queue.put(req)

    while 'output' not in req:
        time.sleep(CHECK_INTERVAL)

    return jsonify(req['output'])


@app.route('/healthz', methods=["GET"])
def health_check():
    return "Health", 200

## Server 실행

Flask 명령으로 서버를 실행합니다.


In [ ]:
app.run(port=5000, host='127.0.0.1')

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
